In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import random

import pickle

from bs4 import BeautifulSoup
import requests
import lxml
import unicodedata

## Scrape current-season NBA stats
 
* This should be done daily 

In [38]:
url = 'https://www.basketball-reference.com/leagues/NBA_2022_per_game.html'
def scrape_current_season_stats(url):
    '''Get current season stats for all players from basketball-reference.com'''
    d = {}
  
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    df = pd.read_html(str(soup.find(id='all_per_game_stats')))[0]
    return df


test = scrape_current_season_stats(url)
print(test.columns)
test.head()

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')


,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C,22,TOR,21,17,26.5,3.4,8.8,...,.543,2.3,5.9,8.2,1.6,0.5,0.6,1.1,2.5,8.0
1,2,Steven Adams,C,28,MEM,29,29,24.9,2.6,4.9,...,.619,3.9,4.9,8.8,2.6,0.9,0.6,1.8,1.7,7.0
2,3,Bam Adebayo,C,24,MIA,18,18,32.9,7.0,13.5,...,.759,2.7,7.4,10.2,3.2,1.1,0.3,2.9,3.3,18.7
3,4,Santi Aldama,PF,21,MEM,16,0,9.8,1.5,4.1,...,.583,1.0,1.6,2.6,0.8,0.1,0.2,0.3,1.1,3.6
4,5,LaMarcus Aldridge,C,36,BRK,25,8,23.6,6.0,10.4,...,.833,1.4,4.3,5.7,0.9,0.4,1.2,0.8,1.7,14.0


In [39]:
def clean(df_raw):
    df = df_raw.copy()
    
    #Remove spurious rows
    df = df[df['Rk'].astype(str).str.isdigit()]
    df = df.set_index('Rk')
    
    #Normalize names
    df['Player'] = df['Player'].apply(strip_accents_and_periods)

    df['FG%'].fillna(df['FG%'].median(), inplace=True)  # no FG: assume median
    df['FT%'].fillna(df['FT%'].median(), inplace=True)  # no FT: assume median
    df['3P%'].fillna(0, inplace=True)                   # no threes: assume the worst
    
    cols = ['PTS','AST','Age','G','GS','MP','3P','3PA','FT','FTA','ORB', 'TRB', 'AST', 'STL', 'BLK',
            'TOV', 'PTS','3P%','FT%','FG%']
    for col in cols:
        df[col] = df[col].apply(lambda s: pd.to_numeric(s, errors='coerce'))
        
    return df


def strip_accents_and_periods(text):
    '''Normalize player name spellings'''
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(text).replace('.','')


In [40]:
url = 'https://www.basketball-reference.com/leagues/NBA_2022_per_game.html'
dfcurrent = scrape_current_season_stats(url)
dfcurrent = clean(dfcurrent)

dfcurrent.to_csv(r'/Users/andrei/Dropbox/Metis/7-Engineering/andrei-eng-project/data/df_current_clean.csv')